# Decoading Streategy

All a LLM do while generative text is predict the next word. To Predict the next word there are 3 major techinque:
1. Gready Search
2. Beam Search

## Gready Search

 In this approch we select the word with highest probability. Let's try to implement the approch

In [1]:
# Importing necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

model_name = 'gpt2-medium'

# loading tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)  

c:\Users\Karmakar\Desktop\Course\AI\DecoadingStrategy\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
# Defining a function to finf the next word using gready search
def get_next_word_using_gready_search(input_text, input_ids, time_steps, choices_per_step):
    """
    Predict next word using Gready Search.

    Args:
        input_text (str): Input text Sequence.
        input_ids (torch.Tensor): Input Tensor containing tensorized text sequence
        time_steps (int): Number of next steps to predict
        choices_per_step (int): Number of choice at each step.
    
    """

    iterations = []

    with torch.no_grad():
        for _ in range(time_steps):
            iteration = dict()
            iteration['input'] = tokenizer.decode(input_ids[0])
            
            # Predicting using model
            output = model(input_ids = input_ids)
            next_token_logits = output.logits[0,-1,:]
            next_token_probability = torch.softmax(next_token_logits, dim=-1)

            sorted_index_of_next_probability = torch.argsort(next_token_probability, descending=True, dim=-1)

            # Top few highest tokens
            for choics_idx in range(choices_per_step):
                token_index_sorted = sorted_index_of_next_probability[choics_idx]
                token_prob = next_token_probability[token_index_sorted].cpu().numpy()
                token_choice = f"{tokenizer.decode(token_index_sorted)} ({token_prob*100:.2f}%)"
                iteration[f'Choice {choics_idx+1}'] = token_choice
            
            # Appending predicted next token to input
            input_ids = torch.cat([input_ids, sorted_index_of_next_probability[None, 0, None]], dim=-1)
            # iterations.append(iteration)

        # Returning iterations as df
        # return pd.DataFrame(iterations)
        return tokenizer.decode(input_ids[0])

In [21]:
input_text = "Today is a"
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

generated_text = get_next_word_using_gready_search(input_text=input_text,
                                  input_ids=input_ids,
                                  time_steps=40,
                                  choices_per_step=5)
print(generated_text)

Today is a good day for the United States," he said. "We're going to be able to continue to work with our allies and partners to make sure that we're able to continue to make progress on the


Lets generate Gready search result from transformer

In [9]:
# Input Tokens
input_tokens = tokenizer(input_text, return_tensors= 'pt').to(device)

# Generating data from model using Hugging face library
output = model.generate(**input_tokens, max_new_tokens=40)
print(tokenizer.decode(output[0], skip_special_tokens= True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Today is a beautiful day. I'm going to go to the beach and enjoy the sun. I'm going to go to the beach and enjoy the sun. I'm going to go to the beach and enjoy the sun. I


## Beam Search

This is also a deterministic search. Here instead of choosing the word with heighest probability we choose n words with heighest conditional probability.

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load model and tokenizer
model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Beam search function
def beam_search(prompt, max_length=50, num_beams=5):
    model.eval()
    
    # Tokenize input and move to device
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    
    # Initialize beams: Each beam starts with the same initial input
    beams = [(input_ids, 0)]  # Tuple of (tokens, score)
    
    for _ in range(max_length):
        new_beams = []
        # new_beams = beams
        
        for tokens, score in beams:
            # Get model predictions
            with torch.no_grad():
                outputs = model(tokens)
            
            # Extract logits for the last token and apply softmax
            logits = outputs.logits[:, -1, :]
            probs = torch.softmax(logits, dim=-1)
            
            # Get top beam candidates
            top_probs, top_indices = probs.topk(num_beams)
            
            # Create new beams
            for i in range(num_beams):
                new_token = top_indices[:, i].unsqueeze(-1)
                new_score = score + torch.log(top_probs[:, i])  # Update score
                
                new_beam = (torch.cat([tokens, new_token], dim=-1), new_score)
                new_beams.append(new_beam)
        
        # Sort beams by score and keep the best ones
        new_beams.sort(key=lambda x: x[1], reverse=True)
        beams = new_beams[:num_beams]
    
    # Select the best final beam
    best_tokens = beams[0][0]
    return tokenizer.decode(best_tokens[0], skip_special_tokens=True)

# Example usage
input_text = "i love"
generated_text = beam_search(input_text, max_length=2, num_beams=3)
print(generated_text)

i love it.


Trying Hugging face library to predict next word using beam search

In [7]:
input_text = "i love"

# Input Tokens
input_tokens = tokenizer(input_text, return_tensors= 'pt').to(device)

# Generating data from model using Hugging face library
output = model.generate(**input_tokens, max_new_tokens=2, num_beams = 3)
print(tokenizer.decode(output[0], skip_special_tokens= True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


i love it.
